In [1]:
import mlflow
from mlflow.models import infer_signature

import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
# locoalhost:5000
mlflow.set_tracking_uri('http://localhost:5002')
#set your experiment name here
mlflow.set_experiment('First-experiment')

2025/09/02 18:24:45 INFO mlflow.tracking.fluent: Experiment with name 'First-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/523376314757050689', creation_time=1756830285293, experiment_id='523376314757050689', last_update_time=1756830285293, lifecycle_stage='active', name='First-experiment', tags={}>

- Load and prepare the Iris dataset for modeling.

- Train a Logistic Regression model and evaluate its performance.

- Prepare the model hyperparameters and calculate metrics for logging

In [4]:
# Load the Iris dataset
X, y = datasets.load_iris(return_X_y=True)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# Define the model hyperparameters
params = {
    "solver": "lbfgs",
    "max_iter": 1000,
    "multi_class": "auto",
    "random_state": 8888,
}

In [6]:
# Train the model
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

# Predict on the test set
y_pred = lr.predict(X_test)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)

/Users/bilo/Documents/Ecole/Campus DevOps/MLflow-Workshop/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


### Log the model and hyperparameters to MLflow:

- Initiate an MLflow run context to start a new run that we will log the model and metadata to.

- Log model parameters and performance metrics.

- Tag the run for easy retrieval.

- Register the model in the MLflow Model Registry while logging (saving) the model.

In [7]:
# Start an MLflow run
with mlflow.start_run():
    # Log the hyperparameters
    mlflow.log_params(params)

    # Log the loss metric
    mlflow.log_metric("accuracy", accuracy)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic LR model for iris data")

    # Infer the model signature
    signature = infer_signature(X_train, lr.predict(X_train))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="iris_model",
        signature=signature,
        input_example=X_train,
        registered_model_name="tracking-quickstart",
    )

2025/09/02 18:24:55 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
Successfully registered model 'tracking-quickstart'.
2025/09/02 18:24:59 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking-quickstart, version 1


🏃 View run bouncy-loon-916 at: http://localhost:5002/#/experiments/523376314757050689/runs/4989f7c6d0be4ff98848f697f1bb71de
🧪 View experiment at: http://localhost:5002/#/experiments/523376314757050689


Created version '1' of model 'tracking-quickstart'.


### Load the Model and Test it

In [8]:
# Load as PyFuncModel for Pandas DataFrame
model_uri = "runs:/4989f7c6d0be4ff98848f697f1bb71de/log_reg_model"  # <--- EXEMPLE, UTILISEZ LA VÔTRE !

# Charger le modèle
loaded_model = mlflow.pyfunc.load_model(model_uri)

print("Modèle chargé avec succès !")

KeyboardInterrupt: 

In [9]:
predictions = loaded_model.predict(X_test)

In [ ]:
iris_feature_names = datasets.load_iris().feature_names

result = pd.DataFrame(X_test, columns=iris_feature_names)
result["actual_class"] = y_test
result["predicted_class"] = predictions

result[:4]